# FLG Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "FLG"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
DSG3
TGM1
DSG1
DSC1
PKP1
CSTA
ST14
EVPL
SPRR1B
SPRR3
KLK5
PPL
DSC3
LOR
SPRR2E
SPRR2B
SPRR2D
SPRR1A
IVL
FLG
PRSS8
DSP
SPRR2A
CASP14
TCHH
PCSK6
USP1
UCHL5
CUL2
MPG
TUBG1
EZH2
C21orf59
UCP2
GPR18
KIR3DS1
MCL1
GMCL1
HNRNPA1
TNFRSF19
FCF1
PI4KA
FTH1
STK16
ZSCAN12
MAK
CYLD
HTN3


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

                               Comparison   P_Value
0  HTN3_transcriptomics_ENSG00000205649.6  0.000061





# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  FLG


Doing t-test comparisons

                                          Comparison       P_Value
0    RP11-249L21.2_transcriptomics_ENSG00000218872.1  1.807788e-10
1      CTC-369M3.1_transcriptomics_ENSG00000253638.1  5.642627e-10
2   RP11-1028N23.2_transcriptomics_ENSG00000258280.1  7.857995e-10
3         RN7SL42P_transcriptomics_ENSG00000263999.2  8.538047e-08
4    CTD-2537I9.15_transcriptomics_ENSG00000279063.1  9.486508e-08
5       AC006380.3_transcriptomics_ENSG00000231418.1  1.290821e-07
6        PPEF1-AS1_transcriptomics_ENSG00000237221.1  1.338733e-07
7            Y_RNA_transcriptomics_ENSG00000199912.1  3.315852e-07
8     RP11-321L2.3_transcriptomics_ENSG00000277608.1  3.333389e-07
9     RP11-497J7.2_transcriptomics_ENSG00000233880.1  3.333389e-07
10         DEFB131_transcriptomics_ENSG00000186146.1  3.390976e-07
11    RP11-552O4.2_transcriptomics_ENSG00000266184.1  5.084511e-07
12   RP11-109P11.1_transcriptomics_ENSG00000260183.1  5.746480e-07
13           Y_RNA_transcriptomics_E

# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,FLG,HTN3_transcriptomics_ENSG00000205649.6,True,6.119048e-05
0,Gbm,FLG,RP11-249L21.2_transcriptomics_ENSG00000218872.1,False,1.807788e-10
1,Gbm,FLG,CTC-369M3.1_transcriptomics_ENSG00000253638.1,False,5.642627e-10
2,Gbm,FLG,RP11-1028N23.2_transcriptomics_ENSG00000258280.1,False,7.857995e-10
3,Gbm,FLG,RN7SL42P_transcriptomics_ENSG00000263999.2,False,8.538047e-08
4,Gbm,FLG,CTD-2537I9.15_transcriptomics_ENSG00000279063.1,False,9.486508e-08
5,Gbm,FLG,AC006380.3_transcriptomics_ENSG00000231418.1,False,1.290821e-07
6,Gbm,FLG,PPEF1-AS1_transcriptomics_ENSG00000237221.1,False,1.338733e-07
7,Gbm,FLG,Y_RNA_transcriptomics_ENSG00000199912.1,False,3.315852e-07
8,Gbm,FLG,RP11-321L2.3_transcriptomics_ENSG00000277608.1,False,3.333389e-07


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"